In [1]:
import os
import cv2
import tqdm
import numpy as np

In [21]:
def transfer_files(root_folder, output_folders, dataset_name, filter_function=None,
                   mask_as_jpeg=False):
    listFolders = os.listdir(root_folder)
    attributionMethods = [l for l in listFolders if l !='images']
    imageFolder = os.path.join(root_folder, 'images')
    listImages = os.listdir(imageFolder)
    listImages = [_ for _ in listImages if _.endswith('.jpeg')]
    if filter_function:
        listImages = filter_function(listImages)
    for file in tqdm.tqdm(listImages):
        image = cv2.imread(os.path.join(imageFolder, file))
        h, w, c = image.shape
        imgOutputFolder = os.path.join(output_folders, 'image')
        os.makedirs(imgOutputFolder, exist_ok=True)
        cv2.imwrite(os.path.join(imgOutputFolder, dataset_name+'_'+file), image)
        new_h = 384
        new_w = int(w*(new_h/h))
        for attrFolder in attributionMethods:
            file = file.replace('jpeg', 'png')
            outAttrFolder = os.path.join(output_folders, 'masks', attrFolder)
            os.makedirs(outAttrFolder, exist_ok=True)
            mask_path = os.path.join(root_folder, attrFolder, 'Heatmap', file)
            mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
            mask = cv2.resize(mask, (new_w, new_h))
            if mask_as_jpeg:
                save_file = file.replace('png', 'jpeg')
            else:
                save_file = file
            cv2.imwrite(os.path.join(outAttrFolder, dataset_name+'_'+save_file), mask)

In [22]:
root_folder = '/home/clement/Documents/phd/FocusedAttention/data/interpretabilityMaps/IDRID'
output_folders = '../data/dataset'
dataset_name = 'IDRID'
transfer_files(root_folder, output_folders, dataset_name)


100%|██████████| 27/27 [00:02<00:00, 12.03it/s]


In [23]:
root_folder = '/home/clement/Documents/phd/FocusedAttention/data/interpretabilityMaps/Kermany'
output_folders = '../data/dataset'
dataset_name = 'Kermany'

def filter_function(list_files):
    def r_index(start, stop, n):
        index = np.arange(start, stop)
        np.random.shuffle(index)
        return index[:n]
    indexes = np.concatenate([r_index(250,500, 50), r_index(500,750, 50), r_index(750,1000, 50)])
    return ['sample_%i.jpeg'%i for i in indexes]

transfer_files(root_folder, output_folders, dataset_name, filter_function,
               mask_as_jpeg=True)



100%|██████████| 150/150 [00:12<00:00, 12.28it/s]
